# Typst Express Docs

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
import sys
# just in case reload_ext does not work
# Remove the modules if they exist in sys.modules
for mod in ["typst_importer.typst_to_svg", "typst_importer.curve_utils", "typst_importer.svg_preprocessing"]:
    if mod in sys.modules:
        print(f"Removing {mod} from sys.modules")
        del sys.modules[mod]

# Now Python 'forgets' those imports. Reimport them:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

ModuleNotFoundError: No module named 'typst_importer'

Info: Cleared System Clipboard


In [6]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

c = typst_express("$ a = b/d$", scale_factor=100, origin_to_char=False)
shift_scene_content(c)  

In [7]:
content = "$ limits(integral)_a^b f(x) dif x $" 
c = typst_express(content, name="Integral Example")
shift_scene_content(c) 

Read blend: "/Users/jan-hendrik/projects/bouning_equation/thumbnail_typst.blend"
Info: Cleared System Clipboard


In [8]:
c = typst_express("""
#let korange() = text(fill: orange)[$k$]
#let nblue() = text(fill: blue)[$n$]
$ sum_(#korange() = 1)^#nblue() #korange() = (nblue()(nblue()+1)) / 2 $  
""")
shift_scene_content(c) 

In [9]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

c = typst_express("""
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex
#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: 2pt + luma(0),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
for i in range(0,10):
  print(i)
```]
""")
shift_scene_content(c, margin=1.2) 

In [10]:
content = """
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex

#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: 5pt + luma(0),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python

class HelloWorld:
    def __init__(self):
        self.message = "Hello, world!"

    def greet(self):
        print(self.message)

if __name__ == "__main__":
    hello = HelloWorld()
    hello.greet()
```]
"""
c = typst_express(content)
shift_scene_content(c) 

In [11]:
header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a = b/c $"
c = typst_express(header+body)
shift_scene_content(c) 

In [12]:
header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ limits(integral)_a^b f(x) dif x $"
c = typst_express(header+body, scale_factor=200, origin_to_char=True)
shift_scene_content(c) 

In [13]:
content = """
#set math.lr(size: 80%)
$ integral.triple _V (nabla dot accent(F, arrow)) dif V = integral.surf_(partial V)  (accent(F, arrow) dot accent(n, arrow)) dif A $
"""
c = typst_express(content)
shift_scene_content(c) 

In [ ]:
n